In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
import csv
from batchmake import Batcher
import pprint
tf.set_random_seed(777)
from tensorflow.python.layers import core

In [2]:
datas= 'data_samples/'
vocabulary = 'vocab.csv'
vocabs = []
with open(vocabulary, 'r', newline='', encoding='utf-8') as vocab:
    words = csv.reader(vocab)
    for i, word in enumerate(words):
        vocabs.append(word[0])
#         vocabs[word[0]]=i
#         vocabs[word] = i
print(len(vocabs))
word_to_id = {word: i for i, word in enumerate(vocabs)}
id_to_word = {i:word for i, word in enumerate(vocabs)}

22673


In [3]:
id_to_word[0], word_to_id['<PAD>']

('<PAD>', 0)

In [14]:
hidden_size=100
sequence=5
embedding_dim=50
attention_size = 50
batch_size=8
vocab_size=len(vocabs)
checkpoint = './seq2seq_test.ckpt'

In [10]:
def source_to_seq(text):
    '''Prepare the text for the model'''
    sequence_length = 5
    return [word_to_id.get(word, word_to_id['<UNK>']) for word in text]+ [word_to_id['<PAD>']]*(sequence_length-len(text))

In [11]:
input_sentence = 'from'
text = source_to_seq([input_sentence])
print(text)

[13, 0, 0, 0, 0]


In [12]:
text = [38,455,19,3138,0]
print(text)
text = [8,0,0,0,0]

[38, 455, 19, 3138, 0]


In [15]:
checkpoint = './seq2seq_test/seq2seq_test.ckpt'
loaded_graph = tf.Graph()

with tf.Session(graph=loaded_graph) as Ses:
    loader = tf.train.import_meta_graph(checkpoint +'.meta')
    loader.restore(Ses, checkpoint)
    
    print('loaded_graph.get_all_collection_keys()',loaded_graph.get_all_collection_keys())
    
    input_data = loaded_graph.get_tensor_by_name('inputs_xdata:0')
    
    predictions_sample_id = loaded_graph.get_tensor_by_name('predictions_sample_id:0')
    
    predictions_output = loaded_graph.get_tensor_by_name('predictions_output:0')
    
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')

    
    print('predictions_sample_id', predictions_sample_id)
    print('predictions_output', predictions_output)
    logits = tf.nn.top_k(predictions_output,2)
    answer_logits = Ses.run([predictions_sample_id,predictions_output, logits], {input_data: [text]*batch_size, 
                                      target_sequence_length: [len(text)]*batch_size, 
                                      source_sequence_length: [len(text)]*batch_size})
    

pad = word_to_id['<PAD>']

print('Original Text:', input_sentence)

print('\nSource')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([id_to_word[i] for i in text])))
# print(answer_logits)

INFO:tensorflow:Restoring parameters from ./seq2seq_test/seq2seq_test.ckpt
loaded_graph.get_all_collection_keys() ['trainable_variables', 'variables', 'model_variables', 'while_context', 'cond_context', 'train_op', 'queue_runners', 'summaries']
predictions_sample_id Tensor("predictions_sample_id:0", shape=(8, ?), dtype=int32)
predictions_output Tensor("predictions_output:0", shape=(8, ?, 22673), dtype=float32)
Original Text: from

Source
  Word Ids:    [8, 0, 0, 0, 0]
  Input Words: return <PAD> <PAD> <PAD> <PAD>


In [16]:
answer_logits

[array([[3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3]]),
 array([[[-7.41781712,  3.0563426 ,  4.2410531 , ..., -7.18319845,
          -7.36086512, -7.27090645]],
 
        [[-7.41781712,  3.0563426 ,  4.2410531 , ..., -7.18319845,
          -7.36086512, -7.27090645]],
 
        [[-7.41781712,  3.0563426 ,  4.2410531 , ..., -7.18319845,
          -7.36086512, -7.27090645]],
 
        ..., 
        [[-7.41781712,  3.0563426 ,  4.2410531 , ..., -7.18319845,
          -7.36086512, -7.27090645]],
 
        [[-7.41781712,  3.0563426 ,  4.2410531 , ..., -7.18319845,
          -7.36086512, -7.27090645]],
 
        [[-7.41781712,  3.0563426 ,  4.2410531 , ..., -7.18319845,
          -7.36086512, -7.27090645]]], dtype=float32),
 TopKV2(values=array([[[ 4.26558352,  4.2410531 ]],

       [[ 4.26558352,  4.2410531 ]],

       [[ 4.26558352,  4.2410531 ]],

       [[ 4.26558352,  4.2410531 ]],

       [[ 4.26558352,  4.2410531 ]],

       [[ 4.265583

In [17]:
answer_logits[0].shape
print(answer_logits[2])
print('\nTarget')
print('  Word Ids:       {}'.format([i for i in answer_logits[0][1] if i != pad]))
print('  Response Words: {}'.format(" ".join([id_to_word[i] for i in answer_logits[0][1] if i != pad])))

TopKV2(values=array([[[ 4.26558352,  4.2410531 ]],

       [[ 4.26558352,  4.2410531 ]],

       [[ 4.26558352,  4.2410531 ]],

       [[ 4.26558352,  4.2410531 ]],

       [[ 4.26558352,  4.2410531 ]],

       [[ 4.26558352,  4.2410531 ]],

       [[ 4.26558352,  4.2410531 ]],

       [[ 4.26558352,  4.2410531 ]]], dtype=float32), indices=array([[[3, 2]],

       [[3, 2]],

       [[3, 2]],

       [[3, 2]],

       [[3, 2]],

       [[3, 2]],

       [[3, 2]],

       [[3, 2]]]))

Target
  Word Ids:       [3]
  Response Words: <EOS>
